In [ ]:
using ForneyLab
using GCV
using ProgressMeter

In [3]:
n_samples = 100
y_data = randn(n_samples)

fg = FactorGraph()

κ1, κ2 = 2.0, 0.5
@RV ω ~ GaussianMeanPrecision(1.0, huge)

@RV s_t ~ Beta(placeholder(:alpha), placeholder(:beta))
@RV b_t ~ Bernoulli(s_t)

f(b) = b*κ1 + (1-b)*κ2
@RV κ ~ Nonlinear{Sampling}(b_t,g=f)
@RV z_t_min ~ GaussianMeanPrecision(placeholder(:mz_t_min),placeholder(:wz_t_min))
@RV z_t ~ GaussianMeanPrecision(z_t_min, 10.0)
@RV x_t_min ~ GaussianMeanPrecision(placeholder(:mx_t_min),placeholder(:wx_t_min))
@RV x_t ~ GaussianControlledVariance(x_t_min, z_t, κ, ω)
@RV y_t ~ GaussianMeanPrecision(x_t, 1.0)
# Data placeholder
placeholder(y_t, :y_t)

q = PosteriorFactorization([z_t_min;z_t],[x_t_min;x_t], κ, ω, s_t; ids=[:Z :X :Κ :Ω :S])
algo = messagePassingAlgorithm(free_energy=true)
src_code = algorithmSourceCode(algo, free_energy=true)

eval(Meta.parse(src_code))

# Define values for prior statistics

m_z = Vector{Float64}(undef, n_samples)
w_z = Vector{Float64}(undef, n_samples)
m_s = Vector{Float64}(undef, n_samples)

m_z_t_min, w_z_t_min = 0.0, 10.0
m_x_t_min, w_x_t_min = 0.0, 10.0

(0.0, 10.0)

In [6]:
m_z_t_min

0.0

In [7]:
n_its = 10
marginals_mf = Dict()
F_mf = zeros(n_its,n_samples)
@showprogress for t in 1:n_samples
    global m_z_t_min, w_z_t_min
    # Prepare data and prior statistics
    data = Dict(:y_t       => y_data[t],
                :mz_t_min => m_z_t_min,
                :wz_t_min => w_z_t_min,
                :mx_t_min => m_x_t_min,
                :wx_t_min => w_x_t_min,
                :alpha => 3.0, :beta => 1.0)

    # Initial recognition distributions
    marginals_mf[:z_t] = ProbabilityDistribution(Univariate, GaussianMeanPrecision, m=m_z_t_min, w=w_z_t_min)
    marginals_mf[:z_t_min] = ProbabilityDistribution(Univariate, GaussianMeanPrecision, m=m_z_t_min, w=w_z_t_min)
    marginals_mf[:b_t] = ProbabilityDistribution(Univariate, Bernoulli, p=0.5)
    marginals_mf[:s_t] = vague(Beta)
    marginals_mf[:κ] = vague(SampleList)
    marginals_mf[:ω] = vague(GaussianMeanPrecision)

    # Execute algorithm
    for i = 1:n_its
        stepX!(data, marginals_mf)
        stepZ!(data, marginals_mf)
        stepΚ!(data, marginals_mf)
        stepS!(data, marginals_mf)
        #stepΩ!(data, marginals_mf)
        F_mf[i,t] = freeEnergy(data, marginals_mf)
    end
    m_z_t_min = ForneyLab.unsafeMean(marginals_mf[:z_t])
    w_z_t_min = ForneyLab.unsafePrecision(marginals_mf[:z_t])
    m_s_t = ForneyLab.unsafeMean(marginals_mf[:s_t])
    m_y_t = y_data[t]

    # Store to buffer
    m_z[t] = m_z_t_min
    w_z[t] = w_z_t_min
    m_s[t] = m_s_t
end


Progress: 100%|█████████████████████████████████████████| Time: 0:00:19


In [8]:
F_mf

10×100 Array{Float64,2}:
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN